import xarray as xr

# Load the NetCDF file
file_path = "LE_v2.2.010_iasi_a-nh3_date_201901.nc"
ds = xr.open_dataset(file_path)

# Print a summary of the dataset
print(ds)

# Optional: list variables and dimensions
print("Variables:", list(ds.variables))
print("Dimensions:", ds.dims)

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

# Load the dataset
ds = xr.open_dataset("LE_v2.2.010_iasi_a-nh3_date_201901.nc")

# Extract variables
lon = ds['longitude'].values
lat = ds['latitude'].values
vcd = ds['vcd'].values

# Plot as scatter map
plt.figure(figsize=(10, 6))
sc = plt.scatter(lon, lat, c=vcd, cmap='viridis', s=5, vmin=0, vmax=np.nanpercentile(vcd, 95))
plt.colorbar(sc, label='VCD [mole/m2]')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('IASI NH₃ Vertical Column Density (Jan 2019)')
plt.grid(True)
plt.tight_layout()
plt.show()

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy.interpolate import griddata

# Load the dataset
ds = xr.open_dataset("LE_v2.2.010_iasi_a-nh3_date_201901.nc")
lon = ds['longitude'].values
lat = ds['latitude'].values
vcd = ds['vcd'].values

# Filter out NaNs
mask = ~np.isnan(vcd)
lon, lat, vcd = lon[mask], lat[mask], vcd[mask]

# Define the regular 0.1° grid
lon_grid = np.arange(np.floor(lon.min()), np.ceil(lon.max()), 0.1)
lat_grid = np.arange(np.floor(lat.min()), np.ceil(lat.max()), 0.1)
lon_mesh, lat_mesh = np.meshgrid(lon_grid, lat_grid)

# Interpolate using griddata
vcd_grid = griddata(
    points=(lon, lat),
    values=vcd,
    xi=(lon_mesh, lat_mesh),
    method='linear'
)

# Plot the interpolated VCD with land boundaries
fig = plt.figure(figsize=(12, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
pc = ax.pcolormesh(
    lon_mesh, lat_mesh, vcd_grid,
    cmap='jet', shading='auto',
    vmin=0, vmax=np.nanpercentile(vcd, 95)
)
plt.colorbar(pc, ax=ax, label='VCD [units?]')

# Add land boundaries and coastlines
ax.add_feature(cfeature.LAND, facecolor='lightgray')
ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax.set_title('Interpolated IASI NH₃ VCD (Jan 2019) on 0.1° Grid')
ax.set_extent([lon.min()-1, lon.max()+1, lat.min()-1, lat.max()+1], crs=ccrs.PlateCarree())
ax.gridlines(draw_labels=True)

plt.tight_layout()
plt.show()

# Replace previous plot section with this enhanced version

fig = plt.figure(figsize=(12, 6))
ax = plt.axes(projection=ccrs.PlateCarree())

# Plot interpolated VCD
pc = ax.pcolormesh(
    lon_mesh, lat_mesh, vcd_grid,
    cmap='jet', shading='auto',
    vmin=0, vmax=np.nanpercentile(vcd, 95)
)
plt.colorbar(pc, ax=ax, label='VCD [units?]')

# 🧭 Add clearer geographic features
ax.add_feature(cfeature.BORDERS, linewidth=0.8, edgecolor='black')  # country borders
ax.add_feature(cfeature.COASTLINE, linewidth=1.0, edgecolor='black')  # coastlines
ax.add_feature(cfeature.LAKES, facecolor='none', edgecolor='blue', linewidth=0.5)
ax.add_feature(cfeature.RIVERS, edgecolor='blue', linewidth=0.3)

# Optional: add land for contrast
ax.add_feature(cfeature.LAND, facecolor='lightgray', alpha=0.3)

# Title and extent
ax.set_title('Interpolated IASI NH₃ VCD (Jan 2019) on 0.1° Grid with Boundaries')
ax.set_extent([lon.min()-1, lon.max()+1, lat.min()-1, lat.max()+1], crs=ccrs.PlateCarree())
ax.gridlines(draw_labels=True, linewidth=0.3, color='gray', linestyle='--')

plt.tight_layout()
plt.show()

In [13]:
import xarray as xr

ds = xr.open_dataset("NH3_Interpolated_IASI_AB_201901_202205_merged_satellite.nc")
print(ds)

ValueError: found the following matches with the input file in xarray's IO backends: ['netcdf4', 'h5netcdf']. But their dependencies may not be installed, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from pathlib import Path

def detect_bounds(nc_path):
    """
    Detect lon/lat extents and time range from a gridded NetCDF.
    Returns
        area_str : str   # "N/W/S/E"   (ERA5 expects this order)
        t0       : str   # first YYYY-MM (inclusive)
        t1       : str   # last  YYYY-MM (inclusive)
    """
    ds = xr.open_dataset(nc_path)

    # ---- 1. Grab coordinates no matter what they’re called --------------
    lon_name = next(n for n in ["lon", "longitude", "x"] if n in ds.coords)
    lat_name = next(n for n in ["lat", "latitude", "y"] if n in ds.coords)
    time_name = next(n for n in ["time", "date"] if n in ds.coords)

    lon = ds[lon_name].values
    lat = ds[lat_name].values
    time = pd.to_datetime(ds[time_name].values)

    # ---- 2. Unwrap longitudes to -180…180 (ERA5 default) -----------------
    lon = ((lon + 180) % 360) - 180   # works for 0–360 or -180–180 input

    # ---- 3. ERA5 bounding box wants N, W, S, E (°) -----------------------
    north = np.round(lat.max(), 2)
    south = np.round(lat.min(), 2)
    west  = np.round(lon.min(), 2)
    east  = np.round(lon.max(), 2)
    area_str = f"{north}/{west}/{south}/{east}"

    # ---- 4. Time range (YYYY-MM) ----------------------------------------
    t0 = time.min().strftime("%Y-%m")
    t1 = time.max().strftime("%Y-%m")

    return area_str, t0, t1


# -------------------------------------------------------------------------
# EXAMPLE: run on your NH₃ file and print a mini cdsapi snippet ----------
# -------------------------------------------------------------------------
nc_file = "NH3_Interpolated_IASI_AB_201901_202205_merged_satellite.nc"
area, t_start, t_end = detect_bounds(nc_file)

print("Detected ERA5 area:", area)
print("First month:", t_start, "  Last month:", t_end)

years  = [str(y) for y in range(int(t_start[:4]), int(t_end[:4]) + 1)]
months = [f"{m:02d}" for m in range(1, 13)]

print(f"""
# ------------------------------------------------------------------
# Quick-start cdsapi template using the detected bounds
# ------------------------------------------------------------------
import cdsapi
c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-single-levels',
    {{
        'product_type': 'reanalysis',
        'variable': ['boundary_layer_height', 'total_precipitation'],
        'year':  {years},
        'month': {months},
        'day':   '01',
        'time':  [f"{{h:02d}}:00" for h in range(24)],
        'area':  '{area}',            # N/W/S/E
        'format': 'netcdf'
    }},
    'era5_blh_tp.nc'
)
""")

ValueError: found the following matches with the input file in xarray's IO backends: ['netcdf4', 'h5netcdf']. But their dependencies may not be installed, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html

In [14]:
#!/usr/bin/env python
"""
Quick-look maps for the interpolated IASI NH₃ product
-----------------------------------------------------

* Reads the 12 km-grid NetCDF
* Loops through every monthly time-stamp
* Writes one PNG per month (≈ 5 s total for 40 frames)
"""

import pathlib
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd

# ---------------------------------------------------------------------
# 1.  CONFIG – edit these two lines if your paths / style differ
# ---------------------------------------------------------------------

OUTDIR   = pathlib.Path("fig_nh3")            # all PNGs land here
OUTDIR.mkdir(exist_ok=True)

# ---------------------------------------------------------------------
# 2.  LOAD DATA (one xarray line does it)
# ---------------------------------------------------------------------
# ds  = xr.open_dataset(DATAFILE)          # dims: time, lat, lon
da  = ds["NH3"]                          # shorthand
lats, lons = ds.lat, ds.lon              # 1-D coordinate vectors

# Choose a common colour scale (0–95 % keeps extremes readable)
vmin, vmax = 0, da.quantile(0.95).item()

# ---------------------------------------------------------------------
# 3.  LOOP & PLOT
# ---------------------------------------------------------------------
for t in da.time:
    stamp = pd.to_datetime(t.values).strftime("%Y-%m")   # e.g. 2019-01
    fig   = plt.figure(figsize=(10, 5), dpi=120)
    ax    = plt.axes(projection=ccrs.PlateCarree())

    pcm = ax.pcolormesh(
        lons, lats, da.sel(time=t),              # data slice
        cmap="jet", shading="auto",
        vmin=vmin, vmax=vmax
    )

    # Niceties
    ax.add_feature(cfeature.LAND, facecolor="lightgray")
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax.set_title(f"IASI NH₃ column • {stamp}", fontsize=14)
    ax.set_extent([lons.min()-1, lons.max()+1,
                   lats.min()-1, lats.max()+1])
    ax.gridlines(draw_labels=True, linewidth=0.3)

    cb = fig.colorbar(pcm, ax=ax, orientation="vertical",
                      label=f"NH₃ ({ds.units})")

    # Save and close
    fig.tight_layout()
    fig.savefig(OUTDIR / f"NH3_{stamp}.png", bbox_inches="tight")
    plt.close(fig)

print(f"✓  Plotted {len(da.time)} frames ➜ {OUTDIR.resolve()}")

✓  Plotted 41 frames ➜ /home/student12/fig_nh3


#!/usr/bin/env python
"""
NH₃ dry-deposition maps from IASI column data (2019-2022)
--------------------------------------------------------
* constant deposition velocity Vd (0.5 cm s⁻¹) and BLH (1 km)
* output:  one PNG per month  +  NetCDF of NH3_dep
"""

import pathlib
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd

# ------------------------------------------------------------------
# 0.  Paths & constants  (tweak here only)
# ------------------------------------------------------------------
OUTDIR   = pathlib.Path("fig_nh3_dep")
OUTDIR.mkdir(exist_ok=True)

# Dry-dep parameters
VD  = 0.005          # m s⁻¹  (0.5 cm s⁻¹)
BLH = 1000           # m     (mixed-layer height)

# Unit helpers
MW_NH3    = 17.031           # g mol⁻¹
AVO       = 6.02214076e23    # molecules mol⁻¹
CM2_TO_M2 = 1e4              # 1 cm² = 1×10⁻⁴ m² ⇒ multiply by 1e4

# ------------------------------------------------------------------
# 1.  Load columns  →  kg NH₃ m⁻²
# ------------------------------------------------------------------
col  = ds["NH3"] * 1e16                       # molecules cm⁻²
mol  = (col * CM2_TO_M2) / AVO                # mol m⁻²
mass = mol * MW_NH3 / 1000                    # kg m⁻²

# ------------------------------------------------------------------
# 2.  Dry-dep flux (kg m⁻² s⁻¹)  →  monthly total (kg m⁻² month⁻¹)
# ------------------------------------------------------------------
flux = (mass / BLH) * VD                      # F = Csurf × Vd
sec_per_month = col.time.dt.days_in_month * 24 * 3600
dep  = flux * sec_per_month
dep.name = "NH3_dep"
dep.attrs.update({
    "long_name": "Estimated dry deposition of NH₃",
    "units":     "kg NH3 m⁻² month⁻¹",
    "method":    f"Vd={VD:.3f} m s⁻¹, BLH={BLH} m"
})

# Optional: save the whole field
dep.to_dataset(name="NH3_dep").to_netcdf("NH3_dry_deposition_2019-2022.nc")

# ------------------------------------------------------------------
# 3.  Plot each month
# ------------------------------------------------------------------
lats, lons = ds.lat, ds.lon
vmin, vmax = 0, dep.quantile(0.95).item()     # common colour scale

for t in dep.time:
    stamp = pd.to_datetime(t.values).strftime("%Y-%m")
    fig   = plt.figure(figsize=(10, 5), dpi=120)
    ax    = plt.axes(projection=ccrs.PlateCarree())

    pcm = ax.pcolormesh(
        lons, lats, dep.sel(time=t),
        cmap="plasma", shading="auto",
        vmin=vmin, vmax=vmax
    )

    ax.add_feature(cfeature.LAND, facecolor="lightgray")
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax.set_title(f"NH₃ dry deposition {stamp}", fontsize=14)
    ax.set_extent([lons.min()-1, lons.max()+1,
                   lats.min()-1, lats.max()+1])
    ax.gridlines(draw_labels=True, linewidth=0.3)

    fig.colorbar(pcm, ax=ax, label=dep.attrs["units"])
    fig.tight_layout()
    fig.savefig(OUTDIR / f"NH3_dep_{stamp}.png", bbox_inches="tight")
    plt.show(fig)

print(f"Plotted {len(dep.time)} deposition maps ⇒ {OUTDIR.resolve()}")

In [5]:
import xarray as xr, numpy as np, pathlib, matplotlib.pyplot as plt

# -------------------------------------------------------------------
# 1.  Load NH3 columns (molecules cm-2 x1e16) and convert to kg m-2
# -------------------------------------------------------------------
ds = xr.open_dataset("NH3_Interpolated_IASI_AB_201901_202205_merged_satellite.nc",
                     engine="h5netcdf")

MW, AVO, CM2_M2 = 17.031, 6.02214076e23, 1e4        # g mol-1, mol-1, cm2→m2
mass = (ds.NH3 * 1e16 * CM2_M2 / AVO) * MW / 1000   # kg NH3 m-2

# -------------------------------------------------------------------
# 2.  Source flux with a 12 h lifetime
# -------------------------------------------------------------------
tau = 12 * 3600                                     # s
secs = ds.time.dt.days_in_month * 86_400            # seconds each month
emission_month = (mass / tau) * secs                # kg m-2 per month

# -------------------------------------------------------------------
# 3.  3.5-year total emission
# -------------------------------------------------------------------
emission_total = emission_month.sum(dim="time")     # kg m-2, Jan-2019→May-2022
emission_total.name = "NH3_emission_3.5yr"

# -------------------------------------------------------------------
# 4.  Quick map
# -------------------------------------------------------------------
vmax = np.nanpercentile(emission_total, 99)         # robust colour bar
fig, ax = plt.subplots(figsize=(7,4.5), dpi=150)
pcm = emission_total.plot(
        ax=ax, cmap="plasma", vmin=0, vmax=vmax,
        cbar_kwargs={"label": "kg NH₃ m⁻² (Jan 2019 – May 2022)"})
ax.set_title("Cumulative NH₃ Emission (τ = 12 h)")
ax.set_xlabel("Longitude"); ax.set_ylabel("Latitude")
path = pathlib.Path("fig_total"); path.mkdir(exist_ok=True)
plt.tight_layout(); fig.savefig(path/"NH3_emission_total_2019-2022.png")
print("✓ wrote fig_total/NH3_emission_total_2019-2022.png")

ValueError: unrecognized engine 'h5netcdf' must be one of your download engines: ['scipy', 'store']. To install additional dependencies, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html

In [10]:
import xarray as xr, numpy as np, pathlib, matplotlib.pyplot as plt

# ---------- 1. open the NH3 column file -------------------------------
ds = xr.open_dataset(
        "NH3_Interpolated_IASI_AB_201901_202205_merged_satellite.nc",
        engine="netcdf4"         # or "h5netcdf" if you prefer
     )

# ---------- 2. column ⇒ mass (kg m-2) --------------------------------
MW, AVO, CM2_M2 = 17.031, 6.02214076e23, 1e4
mass = (ds.NH3 * 1e16 * CM2_M2 / AVO) * MW / 1000             # kg m-2   [oai_citation:7‡xarray](https://docs.xarray.dev/en/stable/getting-started-guide/installing.html?utm_source=chatgpt.com)

# ---------- 3. convert to monthly emission, assume τ = 12 h ----------
tau = 12 * 3600                                               # s   (12 h)   [oai_citation:8‡h5netcdf.org](https://h5netcdf.org/?utm_source=chatgpt.com)
secs = ds.time.dt.days_in_month * 86400                       # s/month
emis_month = (mass / tau) * secs                              # kg m-2 mo-1

# ---------- 4. 3.5-year total (Jan-2019 → May-2022) ------------------
emis_total = emis_month.sum("time")
emis_total.attrs["units"] = "kg NH₃ m⁻² (41 months)"

# ---------- 5. quick map ---------------------------------------------
path = pathlib.Path("fig_total"); path.mkdir(exist_ok=True)
vmax = np.nanpercentile(emis_total, 99)
fig, ax = plt.subplots(figsize=(7,4.5), dpi=150)
pcm = emis_total.plot(
        ax=ax, cmap="plasma", vmin=0, vmax=vmax,
        cbar_kwargs={"label": emis_total.attrs["units"]})
ax.set_title("Cumulative NH₃ Emission (Jan 2019 – May 2022, τ = 12 h)")
ax.set_xlabel("Longitude"); ax.set_ylabel("Latitude")
fig.tight_layout(); fig.savefig(path/"NH3_emission_total_2019-2022.png")
print("✓ wrote fig_total/NH3_emission_total_2019-2022.png")

ValueError: unrecognized engine 'netcdf4' must be one of your download engines: ['scipy', 'store']. To install additional dependencies, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html